In [9]:
from selenium import webdriver
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time
from selenium.webdriver.common.keys import Keys
import re
import csv

# Getting clubs data

In [10]:
driver_path="/usr/local/bin/chromedriver"
driver = webdriver.Chrome(driver_path)

/var/folders/cv/t6swlb9d2sgbcffwwm1rdpz00000gn/T/ipykernel_29463/2124719620.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(driver_path)


In [136]:
wepage_path="https://www.premierleague.com"
#driver.get(wepage_path+"/clubs")

* ### Getting references of clubs dirs

In [103]:

#content = driver.page_source
refrences=[]


content=requests.get(wepage_path+"/clubs")
soup = BeautifulSoup(content.text, 'html.parser')
for a in soup.findAll('a',href=True, attrs={'class':'indexItem'}):
    stad= a.find(class_="stadiumName")
    refrences.append([wepage_path+a['href'],stad.text])

refrences

[['https://www.premierleague.com/clubs/1/Arsenal/overview',
  'Emirates Stadium'],
 ['https://www.premierleague.com/clubs/2/Aston-Villa/overview', 'Villa Park'],
 ['https://www.premierleague.com/clubs/130/Brentford/overview',
  'Brentford Community Stadium'],
 ['https://www.premierleague.com/clubs/131/Brighton-and-Hove-Albion/overview',
  'Amex Stadium'],
 ['https://www.premierleague.com/clubs/43/Burnley/overview', 'Turf Moor'],
 ['https://www.premierleague.com/clubs/4/Chelsea/overview', 'Stamford Bridge'],
 ['https://www.premierleague.com/clubs/6/Crystal-Palace/overview',
  'Selhurst Park'],
 ['https://www.premierleague.com/clubs/7/Everton/overview', 'Goodison Park'],
 ['https://www.premierleague.com/clubs/9/Leeds-United/overview',
  'Elland Road'],
 ['https://www.premierleague.com/clubs/26/Leicester-City/overview',
  'King Power Stadium'],
 ['https://www.premierleague.com/clubs/10/Liverpool/overview', 'Anfield'],
 ['https://www.premierleague.com/clubs/11/Manchester-City/overview',
  

* ### Getting club stadium

In [157]:
import re

def get_club_stadium(club_path):


    link = club_path[0].replace('overview','stadium')
    print(link)
    driver.get(link)
   

    content=driver.page_source
    print(club_path[0]+"stadium")
    #print(content.text)
    bs= BeautifulSoup(content,'html.parser')
    x= bs.find('div', attrs={'data-ui-tab':'Stadium Information'})
    #print(x)
    try:
        capacity= x.find('strong', text="Capacity:").next_sibling
        capacity=capacity.text
    except:
        capacity= None
    
    try:
        Record_attendance= x.find('strong', text="Record PL attendance:").next_sibling
        Record_attendance=Record_attendance.text
    except:
        Record_attendance= None
    try:
        Built_date= x.find('strong', text=re.compile("Built:|Opened:")).next_sibling
        Built_date=Built_date.text
    except:
        Built_date= None

    try:
        Pitch_size= x.find('strong', text="Pitch size:").next_sibling
        Pitch_size=Pitch_size.text
    except:
        Pitch_size= None
    try:
        address= x.find('strong', text="Stadium address:").next_sibling
        address=address.text
    except:
        address= None

    #print(capacity, Record_attendance, Built_date,Pitch_size,address)
    return [capacity, Record_attendance, Built_date,Pitch_size,address]

* ### Getting clubs data

In [ ]:
def get_club_data(club_path):
    #print(club_path[0])
    content=requests.get(club_path[0])
    bs= BeautifulSoup(content.text,'html.parser')
    #print(club_path)
    website= bs.find('div',class_="website")
    #print(website)
    website= website.a.text
    stadium_name= club_path[1]
    name= bs.find(class_="team js-team")
    name= name.text
    #print(name, website, stadium_name)
    club_row= [name,website, stadium_name]
    stadium_row=[stadium_name]+ get_club_stadium(club_path)

    return club_row, stadium_row

In [160]:
#refrences=[['https://www.premierleague.com/clubs/1/Arsenal/',
#  'Emirates Stadium']]
club_list=[["club_name","website", "stadium_name"],]
stadium_list=[["stadium_name","capacity", "Record_attendance", "Built_date","Pitch_size","address"],]
for refrence in refrences:
    one, two=get_club_data(refrence)
    club_list.append(one)
    stadium_list.append(two)

f = open('clubs.csv', 'w')
writer = csv.writer(f)
for row in club_list:
    writer.writerow(row)
f.close()
f = open('stadiums.csv', 'w')
writer = csv.writer(f)
for row in stadium_list:
    writer.writerow(row)
f.close()

Arsenal www.arsenal.com Emirates Stadium
https://www.premierleague.com/clubs/1/Arsenal/stadium
https://www.premierleague.com/clubs/1/Arsenal/overviewstadium
Aston Villa www.avfc.co.uk Villa Park
https://www.premierleague.com/clubs/2/Aston-Villa/stadium
https://www.premierleague.com/clubs/2/Aston-Villa/overviewstadium
Brentford www.brentfordfc.com Brentford Community Stadium
https://www.premierleague.com/clubs/130/Brentford/stadium
https://www.premierleague.com/clubs/130/Brentford/overviewstadium
Brighton and Hove Albion www.brightonandhovealbion.com Amex Stadium
https://www.premierleague.com/clubs/131/Brighton-and-Hove-Albion/stadium
https://www.premierleague.com/clubs/131/Brighton-and-Hove-Albion/overviewstadium
Burnley www.burnleyfootballclub.com Turf Moor
https://www.premierleague.com/clubs/43/Burnley/stadium
https://www.premierleague.com/clubs/43/Burnley/overviewstadium
Chelsea www.chelseafc.com Stamford Bridge
https://www.premierleague.com/clubs/4/Chelsea/stadium
https://www.premi

* ### Getting players

In [209]:

def get_player_rest(player_link, name, nationality, position):
    #here
    print("here")
    print(player_link)
    player_link="https:"+player_link
    content=requests.get(player_link)
    bs= BeautifulSoup(content.text,'html.parser')
    date=None
    height=None
    club_season_21_22=None
    club_season_20_21=None
    club_season_19_20=None
    club_season_18_19=None
    try:
        date= bs.find(class_="pdcol2").find(class_="info")
        date=date.text
        date=re.sub(r"[\n\t\s]*", "", date)
    except:
        #print(player_link+"has no date")
         date=None    

    try:
        height= bs.find(class_="pdcol3").find(class_="info")
        height=height.text
    
    except:
        #print(player_link+"has no height")
        height=None


    for x in bs.find_all(class_="table"):
            try:
                s=x.find(class_="season")
                s=s.text
                #print(s)
                if ("2021/2022" in s):
                    cl= x.find('span', class_="long")
                    cl= cl.text
                    club_season_21_22= cl
                elif ("2020/2021" in s):
                    cl= x.find('span', class_="long")
                    cl= cl.text
                    club_season_20_21= cl
                elif ("2019/2020" in s):
                    cl= x.find('span', class_="long")
                    cl= cl.text                   
                    club_season_19_20= cl
                elif ("2018/2019" in s):
                    cl= x.find('span', class_="long")
                    cl= cl.text                    
                    club_season_18_19= cl
            except:
                continue
    
    f = open('played_for.csv', 'a')
    writer = csv.writer(f)
    
 

    if (club_season_21_22 != None):
        row = [name, nationality, position,"2021/2022", club_season_21_22 ]
        writer.writerow(row)

    if (club_season_20_21 != None):
        row = [name, nationality, position,"2020/2021", club_season_20_21 ]
        writer.writerow(row)

    if (club_season_19_20 != None):
        row = [name, nationality, position,"2019/2020", club_season_19_20 ]
        writer.writerow(row)

    if (club_season_18_19 != None):
        row = [name, nationality, position,"2018/2019", club_season_18_19 ]
        writer.writerow(row)
    
    f.close()

    #print(club_season_21_22, club_season_20_21, club_season_19_20, club_season_18_19)

    return date,height


def get_player_info():
    driver_path="/usr/local/bin/chromedriver"
    driver = webdriver.Chrome(driver_path)
    players_path='https://www.premierleague.com'
    driver.get(players_path+'/players')

    time.sleep(3)

    elem = driver.find_element_by_tag_name("body")
    no_of_pagedowns = 30

    while no_of_pagedowns:
        elem.send_keys(Keys.PAGE_DOWN)
        elem.send_keys(Keys.PAGE_DOWN)
        elem.send_keys(Keys.PAGE_DOWN)
        time.sleep(2.5)
        no_of_pagedowns-=1

    
    content=driver.page_source
    bs= BeautifulSoup(content,'html.parser')
    #print(bs.findAll(class_='dataContainer indexSection'))
    for x in bs.find("tbody").find_all("tr"):
        try:
            player_name= x.find(class_="playerName")
            player_name=player_name.text
        except:
            player_name=None

            
        try:
            position=x.find(class_="hide-s")
            position=position.text
        except:
            position=None

        try:
            nationality=x.find(class_="playerCountry")
            nationality=nationality.text
        except:
            nationality=None
        try:
            player_link= x.a['href']
            player_link=player_link
        except:
            player_link=None
        date,height = get_player_rest(player_link, player_name,nationality, position)
        player_row= [player_name,position,nationality,date,height]
        f = open('players.csv', 'a')
        writer = csv.writer(f)

        writer.writerow(player_row)
        
        f.close()

    return


player_row= ["player_name","position","nationality","birth_date","height"]

f = open('players.csv', 'w')
writer = csv.writer(f)
writer.writerow(player_row)
f.close()



row = ["name", "nationality", "position","season", "club" ]
f = open('played_for.csv', 'w')
writer = csv.writer(f)
writer.writerow(row)
f.close()

get_player_info()


/var/folders/cv/t6swlb9d2sgbcffwwm1rdpz00000gn/T/ipykernel_92323/916460487.py:85: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(driver_path)
/var/folders/cv/t6swlb9d2sgbcffwwm1rdpz00000gn/T/ipykernel_92323/916460487.py:91: DeprecationWarning: find_element_by_tag_name is deprecated. Please use find_element(by=By.TAG_NAME, value=name) instead
  elem = driver.find_element_by_tag_name("body")


here
//www.premierleague.com/players/48285/Aaron-Ramsey/overview
here
//www.premierleague.com/players/19970/Max-Aarons/overview
here
//www.premierleague.com/players/13279/Abdul-Rahman-Baba/overview
here
//www.premierleague.com/players/13286/Tammy-Abraham/overview
here
//www.premierleague.com/players/9199/Adam-Armstrong/overview
here
//www.premierleague.com/players/10905/Che-Adams/overview
here
//www.premierleague.com/players/72541/Omotayo-Adaramola/overview
here
//www.premierleague.com/players/4852/Adrián/overview
here
//www.premierleague.com/players/4328/Sergio-Agüero/overview
here
//www.premierleague.com/players/90665/Derek-Agyakwa/overview
here
//www.premierleague.com/players/4183/Ahmed-El-Mohamady/overview
here
//www.premierleague.com/players/71432/Rayan-Aït-Nouri/overview
here
//www.premierleague.com/players/20113/Kristoffer-Ajer/overview
here
//www.premierleague.com/players/4499/Nathan-Aké/overview
here
//www.premierleague.com/players/3564/Marc-Albrighton/overview
here
//www.prem

# Getting matches

In [11]:
#https://www.premierleague.com/results

#matchFixtureContainer

def get_matches():
    link_list=[]
    path="https://www.premierleague.com/results"
    driver.get(path)

    time.sleep(3)
    #time.sleep(1)

    elem = driver.find_element_by_tag_name("body")
    no_of_pagedowns = 20

    while no_of_pagedowns:
        elem.send_keys(Keys.PAGE_DOWN)
        elem.send_keys(Keys.PAGE_DOWN)
        elem.send_keys(Keys.PAGE_DOWN)
        time.sleep(2)
        no_of_pagedowns-=1
    
    content=driver.page_source

    bs= BeautifulSoup(content,'html.parser')

    for x in bs.find_all('div' , class_ = 'fixture postMatch'):
        link= x.get('data-href')
        #link= link.split('/')
        link_list.append("https:"+link)
        print("https:"+link, "\n")


    return link_list
link_list=get_matches()

/var/folders/cv/t6swlb9d2sgbcffwwm1rdpz00000gn/T/ipykernel_29463/1297574255.py:13: DeprecationWarning: find_element_by_tag_name is deprecated. Please use find_element(by=By.TAG_NAME, value=name) instead
  elem = driver.find_element_by_tag_name("body")


https://www.premierleague.com/match/66685 

https://www.premierleague.com/match/66683 

https://www.premierleague.com/match/66688 

https://www.premierleague.com/match/66690 

https://www.premierleague.com/match/66686 

https://www.premierleague.com/match/66682 

https://www.premierleague.com/match/66687 

https://www.premierleague.com/match/66689 

https://www.premierleague.com/match/66691 

https://www.premierleague.com/match/66684 

https://www.premierleague.com/match/66705 

https://www.premierleague.com/match/66677 

https://www.premierleague.com/match/66674 

https://www.premierleague.com/match/66675 

https://www.premierleague.com/match/66676 

https://www.premierleague.com/match/66679 

https://www.premierleague.com/match/66672 

https://www.premierleague.com/match/66678 

https://www.premierleague.com/match/66680 

https://www.premierleague.com/match/66681 

https://www.premierleague.com/match/66673 

https://www.premierleague.com/match/66633 

https://www.premierleague.com/ma

In [14]:

from selenium.webdriver.common.by import By

def getting_match_content(link):

  
  
    driver.get(link)
    #time.sleep(6)
    #cookie =driver.find_element(By.XPATH, '/html/body/div[3]/div/div/div[1]/div[5]/button[1]') 
    
    #cookie.click()
    time.sleep(3)
    stats = driver.find_element(By.XPATH, '//*[@id="mainContent"]/div/section[2]/div[2]/div[2]/div[1]/div/div/ul/li[3]')
    stats.click()
    time.sleep(1.5)
    content=driver.page_source

    return content

   


In [15]:
    
def conte(content, link) :
    bs= BeautifulSoup(content,'html.parser')
    team_home=None
    team_away= None
    goals_home= 0
    goals_away= 0
    Possession_home=0
    Possession_away=0
    shots_home=0
    shots_away=0
    fouls_home=0
    fouls_away=0
    yellow_cards_home=0
    yellow_cards_away=0
    red_cards_home=0
    red_cards_away=0
    result=None
    date=None
    match_number=None
    season= None
    stadium=None

    x= bs.find('div', class_="team home").find('span', class_="long")
    team_home=x.text
    #print(team_home)
    x= bs.find('div', class_="team away").find('span', class_="long")
    team_away=x.text
    #print(team_away)

    x= bs.find('div', class_="score fullTime")
    result=x.text
    #print(result)
    y= result.split("-")
    goals_home= y[0]
    goals_away= y[1]
    #print(goals_home, goals_away)

    season= "2021/22"
    #print(content)
    
    match_number= link.split("/")
    match_number= match_number[-1]
    #print(match_number)

    x= bs.find('div', class_="matchDate renderMatchDateContainer")
    date=x.text
    #print(date)

    x= bs.find('div', class_="stadium")
    stadium=x.text
    stadium=stadium.split(',')
    stadium=stadium[0]
    #print(stadium)
   
    x= bs.find('tbody', class_="matchCentreStatsContainer").find_all("td")

    for i in range (len(x)):
        #print(i)
        #print(type (x[i]))
        if "Possession" in x[i].text:
            #print(x[i].text, x[i-1].text)
            Possession_home= x[i-1].text
            Possession_away = x[i+1].text
        if "Shots" in x[i].text:
            #print(x[i].text, x[i-1].text)
            shots_home = x[i-1].text
            shots_away = x[i+1].text
        if "Fouls" in x[i].text:
            #print(x[i].text, x[i-1].text)
            fouls_home = x[i-1].text
            fouls_away = x[i+1].text
        if "Yellow cards" in x[i].text:
            #print(x[i].text, x[i-1].text)
            yellow_cards_home = x[i-1].text
            yellow_cards_away = x[i+1].text
        if "Red cards" in x[i].text:
            #print(x[i].text, x[i-1].text)
            red_cards_home = x[i-1].text
            red_cards_away = x[i+1].text       


    match_row=[match_number, season,stadium,  result,date]
    attended_row_home=[team_home, match_number,goals_home,Possession_home, shots_home,fouls_home,yellow_cards_home,red_cards_home,team_away,goals_away,Possession_away, shots_away,fouls_away,yellow_cards_away,red_cards_away]
    #attended_row_away=[team_away ,match_number,goals_away,Possession_away, shots_away,fouls_away,yellow_cards_away,red_cards_away]

    f = open('matches.csv', 'a')
    writer = csv.writer(f)
    writer.writerow(match_row)
    f.close()

    f = open('attended.csv', 'a')
    writer = csv.writer(f)
    writer.writerow(attended_row_home)
    #writer.writerow(attended_row_away)
    f.close()



    return 



In [16]:
#link_list= ['https://www.premierleague.com/match/66654']

driver_path="/usr/local/bin/chromedriver"
driver = webdriver.Chrome(driver_path)

match_row=["match_number", "season","stadium",  "result","date"]
attended_row_home=["team_home", "match_number", "goals_home","Possession_home", "shots_home","fouls_home","yellow_cards_home","red_cards_home","team_away","goals_away","Possession_away", "shots_away","fouls_away","yellow_cards_away","red_cards_away"]
#attended_row_away=["team_away","goals_away","Possession_away", "shots_away","fouls_away","yellow_cards_away","red_cards_away"]

f = open('matches.csv', 'w')
writer = csv.writer(f)
writer.writerow(match_row)
f.close()

f = open('attended.csv', 'w')
writer = csv.writer(f)
writer.writerow(attended_row_home)
f.close()

for link in link_list:
    page=getting_match_content(link)
    conte(page, link)



/var/folders/cv/t6swlb9d2sgbcffwwm1rdpz00000gn/T/ipykernel_29463/2973329113.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(driver_path)
